In [1]:
import numpy as np
import tensorflow as tf

/home/jaidevd/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Stages of a tensorflow program:
# 1. building a computational graph
# 2. Running the graph
# Graph consists of:
# 1. Tensors
# 2. Operations on them

In [2]:
a = tf.constant(3)
b = tf.constant(4)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


In [3]:
sess = tf.Session()
print(sess.run(total))

7


In [4]:
sess.run({'a': a, 'b': b, 'total': total})

{'a': 3, 'b': 4, 'total': 7}

In [5]:
# randomness and session runs
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.88092864 0.4813968  0.19236994]
[0.47674835 0.5552434  0.4753003 ]
(array([1.033556 , 1.9995459, 1.4736121], dtype=float32), array([2.033556, 2.999546, 2.473612], dtype=float32))


In [6]:
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.548602 0.838048 0.809641]
[0.30260503 0.877645   0.4833027 ]
(array([1.020182 , 1.0012441, 1.3513012], dtype=float32), array([2.0201821, 2.001244 , 2.3513012], dtype=float32))


In [7]:
# Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None.
# You run an operation to cause a side-effect, not to retrieve a value.
# Examples of this include the initialization, and training ops demonstrated later.

In [8]:
# Feeding
# As it stands, this graph is not especially interesting because it always produces a constant result.
# A graph can be parameterized to accept external inputs, known as placeholders.
# A placeholder is a promise to provide a value later, like a function argument.

In [9]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [12]:
sess.run(z, feed_dict = {x: 1, y: 2})

3.0

In [13]:
# Datasets
# Essentially larger placeholders

In [14]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [16]:
sess.run(next_item)

array([0, 1], dtype=int32)

In [17]:
sess.run(slices.make_one_shot_iterator().get_next())

array([0, 1], dtype=int32)

In [18]:
# what went wrong?

In [19]:
iterator = slices.make_one_shot_iterator()

In [20]:
sess.run(iterator.get_next())

array([0, 1], dtype=int32)

In [21]:
sess.run(iterator.get_next())

array([2, 3], dtype=int32)

In [22]:
# stateful datasets

In [23]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[ 0.7305771  1.0798568 -1.2002906]
[-0.1559945  -0.28983077 -0.9170402 ]
[0.02274479 1.198826   0.700819  ]
[-0.40099108 -0.67431664  0.31373188]
[0.6552985  0.15650804 0.18931875]
[-1.3873857   0.16693595 -0.12938333]
[1.0733204  0.88237125 2.211844  ]
[ 0.10854642 -0.41335043 -0.45018807]
[ 0.06718046 -0.06147345 -0.59459484]
[-0.854807    0.20689937 -0.92510176]


In [24]:
# Layers - A layer is a set of trainable variables or placeholders

In [60]:
x = tf.placeholder(tf.float32, shape=(None, 13))

In [61]:
layer = tf.layers.Dense(units=1)

In [62]:
y = layer(x)

In [63]:
init = tf.global_variables_initializer()
sess.run(init)

In [105]:
# Training with an Example

In [114]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X = diabetes['data']
Y = diabetes['target']

In [115]:
x = tf.constant(X)
y_true = tf.constant(Y.reshape(-1, 1), dtype=tf.float32)
y = tf.layers.dense(x, units=1)

In [116]:
loss = tf.losses.mean_squared_error(predictions=y, labels=y_true)

In [124]:
init = tf.global_variables_initializer()
sess.run(init)

In [118]:
sess.run(loss)

29081.795

In [119]:
# Training

In [130]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

In [131]:
for i in range(10000):
    _, cur_loss = sess.run((train, loss))
    if i % 1000 == 0:
        print(cur_loss)

3346.2568
3163.4026
3061.2634
2998.5752
2958.7578
2933.0464
2916.2468
2905.1514
2897.7437
2892.7407


In [ ]:
# Exercise: Build a classifier for IRIS, with hints